Домашнее задание

Описание данных - https://archive.ics.uci.edu/ml/datasets/Liver+Disorders

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

In [24]:
import re
import pandas as pd
import numpy as np
data = pd.read_csv("bupa.data", header=None)
data.head(7)

,0,1,2,3,4,5,6
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
5,98,55,13,17,17,0.0,2
6,88,62,20,17,9,0.5,1


In [26]:
df = pd.read_csv(
    'bupa.data', sep=',', index_col=False,
    names=['mcv', 'alkphos', 'sgpt', 'sgot', 'gammagt', 'drinks', 'selector'])

In [27]:
df.head(7)

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
5,98,55,13,17,17,0.0,2
6,88,62,20,17,9,0.5,1


In [28]:
print(data.shape)

(345, 7)


Всего 345 расстройств

Посмотрим на соотношение классов

In [29]:
data.iloc[:, -1].value_counts()

2    200
1    145
Name: 6, dtype: int64

2. сделать feature engineering

In [30]:
pd.DataFrame([df.isna().sum(), df.isnull().sum()], index={'na', 'null'})

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
na,0,0,0,0,0,0,0
null,0,0,0,0,0,0,0


In [32]:
# Изменение целевой переменной
df['selector'] = df['selector'].map({'2': 0, '2': 1})

In [33]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df.iloc[:, :-1])
y = df['selector']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

3. обучить любой классификатор (какой вам нравится)

In [43]:
import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

[15:05:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [42]:
xgb_model = xgboost.XGBClassifier(num_class=7,
                                  learning_rate=0.1,
                                  num_iterations=1000,
                                  max_depth=10,
                                  feature_fraction=0.7, 
                                  scale_pos_weight=1.5,
                                  boosting='gbdt',
                                  metric='multiclass',
                                  eval_metric='mlogloss')

In [41]:
 try: 
     import xgboost 
 except ImportError: 
     pass 
 except: 
     print("xgboost is installed...but failed to load!") 
     pass 

In [68]:
def evaluate_results(y_test, y_predict):
    """ Проверка качества модели с помощью DS-метрик"""
    from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

    f1 = f1_score(y_test, y_predict)
    roc = roc_auc_score(y_test, y_predict)
    precision = precision_score(y_test, y_predict, average='binary')
    recall = recall_score(y_test, y_predict, average='binary')
#     print(f'Classification results: \nf1: {round(f1, 3)}\n roc: {round(roc, 3)}\n precision: {round(precision, 3)}\n recall: {round(recall, 3)}')
    return f1, precision, recall, roc

In [69]:
evaluate_results(y_test, y_predict)

ValueError: Found input variables with inconsistent numbers of samples: [69, 271]

4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [51]:
mod_data = pd.get_dummies(df.copy())
mod_data['selector'].value_counts()

Series([], Name: selector, dtype: int64)

In [53]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 37/145 as positives and unlabeling the rest


In [54]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    308
 1     37
Name: class_test, dtype: int64


37 positives

In [56]:
mod_data.head(10)

,0,1,2,3,4,5,6,class_test
0,85,92,45,27,31,0.0,1,-1
1,85,64,59,32,23,0.0,2,-1
2,86,54,33,16,54,0.0,2,-1
3,91,78,34,24,36,0.0,2,-1
4,87,70,12,28,10,0.0,2,-1
5,98,55,13,17,17,0.0,2,-1
6,88,62,20,17,9,0.5,1,-1
7,88,67,21,11,11,0.5,1,1
8,92,54,22,20,7,0.5,1,-1
9,90,60,25,19,5,0.5,1,-1


5. применить random negative sampling для построения классификатора в новых условиях

In [57]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(37, 8) (37, 8)


In [61]:
use_label_encoder=False

In [62]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

[15:18:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\reshe\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


(0.5448275862068965,
 0.3969849246231156,
 0.8681318681318682,
 0.6007326007326007)

6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [70]:
pd.DataFrame([
    evaluate_results(y_test, y_predict),
    evaluate_results(sample_test.loc[:, 'selector'].values, y_predict_rns)
], columns={'f1', 'precision', 'recall', 'roc'}, index={'Simple xgboost', 'random negative sampling'}).round(2)

ValueError: Found input variables with inconsistent numbers of samples: [69, 271]